# Imports

In [ ]:
import torch
import math
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
from utils import utils, verification, verification_inverse
from Reconstruction import direct_attack, transfer_attack

### Desciption of the local FRS and the inverse model. (Architecture / Loss / Train_set)

- Local FRS:     ResNet-50 / ArcFace            / MS1M-v3
- Inverse model: NbNet     / Pixel and Perceptual / MS1M-v3

In [ ]:
device = torch.device('cuda:0')
# Uncomment if you do not have a GPU:
# device = torch.device('cpu')
local = utils.local_FRS().eval().to(device)
local_inverse = utils.local_inverse().eval().to(device)

###### For checking the performance of our local and inverse models. You can use below code. ######
# dataset_lfw = verification.load_bin('./utils/dataset/lfw.bin', (112, 112))
# local_th = verification.test(dataset_lfw, local, 64, device, 10)
# with torch.no_grad():
#     verification_inverse.test_inverse(dataset_lfw, local, local_th, local_inverse, 64, device, 1)

### Description of open-source Target FRS (Architecture / Loss / Train_set)

- t1: ResNet-100         / CosFace     / Glint360K
- t2: Vision Transformer / CosFace     / MS1M-v3
- t3: ResNet-50          / ArcFace     / WebFace12M
- t4: SFNet-20           / SphereFace2 / VGGFace2
- t5: Inception-ResNet   / Triplet     / CASIA-WebFace

### Attack Scenario
- Direct_Type1 attack  : Face reconstruction a image against a target FRS.
- Direct_Type2 attack  : Face reconstruction a image against a target FRS, and test with a different image of same identity.
- Transfer_Type1 attack: Face reconstruction a image against a target FRS, and test against a different target FRS.
- Transfer_Type2 attack: Face reconstruction a image against a target FRS, and test against a different target FRS with a different image of same identity.

In [ ]:
# Modify here. Target FRS of direct attack.

_target_FRS = 't5' # t1 ~ t5 and AWS
direct_type = 2 # Type 1 or 2 attack
target_id = 310 # Available samples: 310, 760, 2167

In [ ]:
vit = False
imgsize = 112
if _target_FRS !='AWS':
    target_FRS = utils.target_FRS(_target_FRS, device)
    target_FRS.eval().to(device)
    
    if _target_FRS == 't2':
        vit = True
    elif _target_FRS == 't5':
        imgsize = 160
    else:
        vit = False
        imgsize = 112

    # Uncomment to compute the thresholds used for FRS
    # dataset_lfw = verification.load_bin('./utils/dataset/lfw.bin', (imgsize, imgsize))
    # th = verification.test(dataset_lfw, target_FRS, 64, device, 10, vit)
    # th_angle = math.acos((th-2)/(-2))*180/math.pi
    
    # Uncomment to use angles as listed in paper
    if _target_FRS == 't1':
        th_angle = 72.5 # T1 angle
    elif _target_FRS == 't2':
        th_angle = 77.3 # T2 angle
    elif _target_FRS == 't3':
        th_angle = 74.3 # T3 angle
    elif _target_FRS == 't4':
        th_angle = 75.8 # T4 angle
    elif _target_FRS == 't5':
        th_angle = 63.6 # T5 angle
    
else:
    target_FRS='AWS'
    th_angle = 80 # threshold value for AWS (score value)

# Direct Attack

In [ ]:
args = edict()
args.device = device
args.local = local
args.local_inverse = local_inverse
ofs = utils.ofs_load()
ofs = ofs.to(device)
args.ofs = ofs
args.bb_FRS = target_FRS
args.bb_th = th_angle
args.bb_imgsize = imgsize
args.direct_type = direct_type
args.target_id = target_id
args.vit = vit

recon, cos = direct_attack(args)

#plot
targetimg1 = utils.targetimg_type1(imgsize=args.bb_imgsize, target_id=args.target_id)

plt.subplot(1, direct_type+1, 1)
plt.axis('off')
plt.imshow(targetimg1)
plt.title('Target image - Type 1')

plt.subplot(1, direct_type+1, 2)
plt.axis('off')
plt.imshow(recon)
plt.title('Reconstruction')

if direct_type==2:
    targetimg2 = utils.targetimg_type2(imgsize=args.bb_imgsize, target_id=args.target_id)
    plt.subplot(1, 3, 3)
    plt.axis('off')
    plt.imshow(targetimg2)
    plt.title('Target image - Type 2')
    
plt.show()
if target_FRS!='AWS':
    print('Angle distance:', math.acos(cos)*180/math.pi)
    print('Threshold:', args.bb_th)
else:
    print('Confidence score:', cos)
    print('Threshold:', args.bb_th)

# Transfer Attack

In [ ]:
# Adds a target FRS of transfer attack.

_target_FRS_transfer = 't2' # t1 ~ t5 and AWS
transfer_type = 1 # 1 or 2

In [ ]:
transfer_vit = False
imgsize_transfer = 112

if _target_FRS_transfer !='AWS':
    target_FRS_transfer = utils.target_FRS(target_FRS = _target_FRS_transfer).eval().to(device)
            
    if _target_FRS_transfer == 't2':
        transfer_vit = True
    elif _target_FRS_transfer == 't5':
        imgsize = 160
    else:
        transfer_vit = False
        imgsize_transfer = 112

    # Uncomment to compute the thresholds used for FRS
    # dataset_lfw = verification.load_bin('./utils/dataset/lfw.bin', (imgsize, imgsize))
    # th_transfer = verification.test(dataset_lfw, target_FRS_transfer, 64, device, 10, transfer_vit)
    # th_transfer_angle = math.acos((th_transfer-2)/(-2))*180/math.pi
    
    # Uncomment to use angles as listed in paper
    if _target_FRS == 't1':
        th_transfer_angle = 72.5 # T1 angle
    elif _target_FRS == 't2':
        th_transfer_angle = 77.3 # T2 angle
    elif _target_FRS == 't3':
        th_transfer_angle = 74.3 # T3 angle
    elif _target_FRS == 't4':
        th_transfer_angle = 75.8 # T4 angle
    elif _target_FRS == 't5':
        th_transfer_angle = 63.6 # T5 angle
else:
    target_FRS_transfer='AWS'
    th_transfer_angle = 80

In [ ]:
args = edict()
args.device = device
args.local = local
args.local_inverse = local_inverse
args.ofs = utils.ofs_load()
args.bb_FRS = target_FRS
args.bb_th = th_angle
args.bb_imgsize = imgsize
args.direct_type = None
args.transfer_FRS = target_FRS_transfer
args.transfer_th = th_transfer_angle
args.transfer_imgsize = imgsize_transfer
args.transfer_type = transfer_type
args.target_id = target_id
args.vit = vit
args.transfer_vit = transfer_vit

recon, cos = transfer_attack(args)

#plot
targetimg1 = utils.targetimg_type1(imgsize=args.bb_imgsize, target_id=args.target_id)

plt.subplot(1, transfer_type+1, 1)
plt.axis('off')
plt.imshow(targetimg1)
plt.title('Target image - Type 1')

plt.subplot(1, transfer_type+1, 2)
plt.axis('off')
plt.imshow(recon)
plt.title('Reconstruction')

if transfer_type==2:
    targetimg2 = utils.targetimg_type2(imgsize=args.bb_imgsize, target_id=args.target_id)
    plt.subplot(1, 3, 3)
    plt.axis('off')
    plt.imshow(targetimg2)
    plt.title('Target image - Type 2')
    
plt.show()
if target_FRS_transfer!='AWS':
    print('Angle distance:', math.acos(cos)*180/math.pi)
    print('Threshold:', args.transfer_th)
else:
    print('Confidence score:', cos)
    print('Threshold:', args.transfer_th)

# GenOFS
Instead of our pre-generated OFS, you can generate your own OFS.
You should prepare a dataset for extracting face templates to find orthogonal templates.

In [ ]:
from GenOFS import *

# device = torch.device('cuda:0')
# Uncomment if you do not have a GPU:
device = torch.device('cpu')
savepath='./utils/param/ofs.pt'

# For your dataset (ex: MS1MV3)
# dataset = 'your dataset'

# For checking using LFW (can not produce large number of OFS due to the number of contained image in LFW)
dataset = verification.load_bin('./utils/dataset/lfw.bin', (112, 112))
ofs_num = 30  # Modify the number of orthogonal face images that you want to generate.

GenOFS(device, dataset, ofs_num, savepath) # OFS is saved in the form of tensors.

In [ ]:
# Checking almost-orthogonality of OFS.
ofs = torch.load(savepath)
model = utils.local_FRS(device=device) # Modify here if you check the alomost-orthogonality in the different FRS.

orthogonal_check(ofs, model)